In [1]:
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import spotipy.util as util
import pprint
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import matplotlib
from operator import itemgetter

In [2]:
# plotly api key
plotly.tools.set_credentials_file(username='kjsnow11', api_key='CdXOsPqwq2G3aMCiHpjJ')

# spotify creds
client_id = '2d0cc7a4ed9d44a69c9ad358b216dd7e'
client_secret = 'bc85301ff7114dca9f2a195804b16ddc'

In [3]:
# create spotify object
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
# Search functions

# Generic search function, valid search_types = [album , artist, playlist, track]
def get_search_uri(search_term, search_type):
    result = sp.search(search_term, type=search_type, limit=1)
    return result[search_type+'s']['items'][0]['id']

# Only supports artists
def get_artist_uri(artist_search):
    result = sp.search(artist_search, type='artist', limit=1)
    return result['artists']['items'][0]['id']

# Only supports tracks
def get_track_uri(song_search):
    result = sp.search(song_search, type='track', limit=1)
    return result['tracks']['items'][0]['id']

# Search by track and artist (to help avoid conflicts)
def get_track_artist_uri(track, artist):
    qry = "artist:{0} track:{1}".format(artist, track)
    result = sp.search(qry, type='track', limit=1)
    return result['tracks']['items'][0]['id']

# Only supports albums
def get_album_uri(album_search):
    result = sp.search(album_search, type='album', limit=1)
    return result['albums']['items'][0]['id']

# Only supporst playlists
def get_playlist_uri(playlist_search):
    result = sp.search(playlist_search, type='playlist', limit=1)
    return result['playlists']['items'][0]['id']

# Return Song title from URI
def get_song_title(song_uri):
    result = sp.search()

In [89]:
# IMPROVEMENTS
# Error handling (no results returned)
# Multiple results? (limit = 1 right now)
# How to better generalize search functions?

In [5]:
# Get key track features based 
def get_track_key_features(track_uri):
    # Neglecting to return type, uri, track_href, analysis_url. Not needed for analysis
    features_of_interest = [
                        'id',
                        'acousticness', 
                        'danceability', 
                        'duration_ms', 
                        'energy', 
                        'instrumentalness',
                        'key',  
                        'liveness',
                        'loudness',
                        'mode',
                        'speechiness',
                        'tempo',
                        'time_signature',
                        'valence']
    track_audio_features = sp.audio_features(track_uri)
    return {k : track_audio_features[0][k] for k in features_of_interest}

In [25]:
def get_sorted_track_list_by_feature_of_interest(track_to_be_compared_key_features, feature_of_interest, artist_to_compare):
    
    artist_to_compare_uri = get_search_uri(search_term=artist_to_compare, search_type='artist')
    artist_to_compare_top_tracks = sp.artist_top_tracks(artist_to_compare_uri)
    artist_to_compare_top_track_uri_list = [t['id'] for t in artist_to_compare_top_tracks['tracks']]
    
    # Build list of dictionaries with name, id, features
    artist_to_compare_track_dict = []
    for track in artist_to_compare_top_tracks['tracks']:
        track_to_add = {
            "name":     track['name'],
            "id":       track['id'],
            "features": get_track_key_features(track['uri']),
        }
        artist_to_compare_track_dict.append(track_to_add)
        
    # Calculate difference of feature of interest
    #diff_of_feature_of_interest = [abs(track[feature_of_interest] 
    #- track_to_be_compared_key_features[feature_of_interest]) for track in track_to_be_compared_key_features]

    artist_to_compare_track_dict = [dict(track, diff_of_feature_of_interest = abs(track['features'][feature_of_interest] - track_to_be_compared_key_features[feature_of_interest])) for track in artist_to_compare_track_dict]
    
    #[dict(track, diff_of_feature_of_interest=
                                    #abs(track['features'][feature_of_interest] - track_key_features[feature_of_interest])) for track in track_dict]
    
    # Another way to sort Dictionary by key value (but not using difference function!)
    
    #print(sorted(track_dict, key=itemgetter(diff_in_acousticness))) 
    sorted_artist_to_compare_track_dict = sorted(artist_to_compare_track_dict, key=itemgetter('diff_of_feature_of_interest'))
    
    return sorted_artist_to_compare_track_dict

def print_sorted_songs(sorted_artist_to_compare_track_dict):
    for track in sorted_artist_to_compare_track_dict:
        print(track['name'])


In [66]:
# Get URI of track
track = 'Sixty Years On- Live'
artist = 'Brandi Carlile'

# Value to compare from original track
feature_of_interest = 'valence'

artist_to_compare = 'Fleetwood Mac'


In [38]:
# Get song URI
track_uri = get_track_artist_uri(track, artist)
# Get key track audio features
track_key_features = get_track_key_features(track_uri)

# Get track audio analysis (not used atm)
track_audio_analysis = sp.audio_analysis(track_uri)

In [53]:
track_key_features

{'acousticness': 0.906,
 'danceability': 0.443,
 'duration_ms': 223980,
 'energy': 0.117,
 'id': '3OZW7SSM5DzQmeq8WZBVek',
 'instrumentalness': 0,
 'key': 2,
 'liveness': 0.705,
 'loudness': -14.124,
 'mode': 0,
 'speechiness': 0.0402,
 'tempo': 136.589,
 'time_signature': 3,
 'valence': 0.105}

In [67]:
sorted_artist_to_compare_track_dict = get_sorted_track_list_by_feature_of_interest(track_to_be_compared_key_features=track_key_features, feature_of_interest=feature_of_interest, artist_to_compare=artist_to_compare)

In [68]:
# Print song names
print_sorted_songs(sorted_artist_to_compare_track_dict)

Landslide
The Chain - 2004 Remaster
Everywhere - 2017 Remaster
Dreams - 2004 Remaster
Rhiannon
Don't Stop - 2004 Remaster
Go Your Own Way - 2004 Remaster
Gypsy
Little Lies - 2017 Remaster
Never Going Back Again - 2004 Remaster


In [ ]:
# Loop over all key features
#... rank all songs overall